[Reference](https://medium.com/@ccpythonprogramming/mimicking-googles-search-speed-with-sqlite-and-python-7b756ae82732)

# 1. Setting Up a Virtual Environment (venv)

```
# Windows Setup:
python -m venv venv
venv\Scripts\activate

# Linux Setup:
python3 -m venv venv
source venv/bin/activate
```

# 2. Project Structure and Files
```
search_index/
│
├── venv/
├── init_db.py
├── add_document.py
├── search_documents.py
└── data/
    └── search_index.db
```

# 3. Initializing the Database

In [1]:
# init_db.py
import sqlite3
import os

DB_PATH = "data/search_index.db"

def init_db():
    if not os.path.exists("data"):
        os.makedirs("data")

    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Enable Full Text Search (FTS5) for efficient indexing
    cursor.execute('''
        CREATE VIRTUAL TABLE IF NOT EXISTS documents USING FTS5(title, content);
    ''')

    conn.commit()
    conn.close()

if __name__ == "__main__":
    init_db()
    print("Database and table initialized.")

Database and table initialized.


# 4. Adding Documents to the Index

In [2]:
# add_document.py
import sqlite3

DB_PATH = "data/search_index.db"

def add_document(title, content):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
        INSERT INTO documents (title, content) VALUES (?, ?)
    ''', (title, content))

    conn.commit()
    conn.close()
    print(f"Document '{title}' added to index.")

if __name__ == "__main__":
    add_document("Python Basics", "Python is a versatile programming language.")
    add_document("Learning SQLite", "SQLite is a powerful database for small applications.")

Document 'Python Basics' added to index.
Document 'Learning SQLite' added to index.


# 5. Implementing Search Functionality

In [3]:
# search_documents.py
import sqlite3

DB_PATH = "data/search_index.db"

def search_documents(query):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
        SELECT title, content FROM documents WHERE documents MATCH ?
    ''', (query,))

    results = cursor.fetchall()
    conn.close()

    if results:
        for title, content in results:
            print(f"Title: {title}\nContent: {content}\n")
    else:
        print("No matching documents found.")

if __name__ == "__main__":
    search_documents("Python")

Title: Python Basics
Content: Python is a versatile programming language.



# Enhancing the System with Phrase Matching

In [4]:
# search_documents.py (updated)
def search_documents(query):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Wrapping query in quotes to find exact phrase
    formatted_query = f'"{query}"'

    cursor.execute('''
        SELECT title, content FROM documents WHERE documents MATCH ?
    ''', (formatted_query,))

    results = cursor.fetchall()
    conn.close()

    if results:
        for title, content in results:
            print(f"Title: {title}\nContent: {content}\n")
    else:
        print("No matching documents found.")